In [ ]:
# ------------------------------------------------
# fetch by actress list by onejav
# ------------------------------------------------
# fetch_onejav_by_actress_list()

# ------------------------------------------------
# fetch by actress list by javland
# ------------------------------------------------
# fetch_javland_by_actress_list()

# ------------------------------------------------
# fetch movies list by fetch_javtrailers
# ------------------------------------------------
JAVTRAILERS_START_PAGE = 1917
# fetch_javtrailers()

# ------------------------------------------------
# generate full text search metadata for each movie
# ------------------------------------------------
ENABLE_FULLTEXT_LLM = False
# generate_fulltext()

In [ ]:
import json
import logging
import re
import os
import sys
import time
from datetime import datetime
from collections import namedtuple
from concurrent.futures import ALL_COMPLETED, ThreadPoolExecutor, wait
from datetime import datetime
from threading import RLock
from hashlib import sha256
from io import BytesIO
from queue import Queue
from textwrap import dedent
from typing import Generator, List, NamedTuple, Optional, Union, Dict, Any

import bson.json_util
import pymongo
import requests
from bs4 import BeautifulSoup
from bson import ObjectId
from kipp.decorator import timer
from kipp.utils import setup_logger
from minio import Minio
from pymongo import MongoClient


logger = setup_logger("jav")
logger.setLevel(logging.INFO)
# logger.setLevel(logging.DEBUG)

os.environ["TZ"] = "UTC"
os.environ["HTTP_PROXY"] = "http://100.122.41.16:17777"
os.environ["HTTPS_PROXY"] = "http://100.122.41.16:17777"
os.environ["http_proxy"] = "http://100.122.41.16:17777"
os.environ["https_proxy"] = "http://100.122.41.16:17777"
os.environ["NO_PROXY"] = (
    "localhost,127.0.0.1,100.64.0.0/10,192.168.0.0/16,10.0.0.0/8,127.0.0.0/8,snake-carp.ts.net"
)


executor = ThreadPoolExecutor(max_workers=100)


sys.path.append(r"/opt/configs/ramjet")
import prd

# mongodb
mongo = MongoClient(
    f"mongodb://{prd.MONGO_ADMIN_USER}:{prd.MONGO_ADMIN_PASSWD}@{prd.MONGO_HOST}:{prd.MONGO_PORT}",
)
col_actress = mongo["jav"]["actress"]
col_movies = mongo["jav"]["movies"]
col_fulltext = mongo["jav"]["fulltext"]


# add index to db
col_movies.create_index([("name", pymongo.ASCENDING)])
col_actress.create_index([("name", pymongo.ASCENDING)])
col_fulltext.create_index([("word", pymongo.ASCENDING)], unique=True)

# minio
s3cli: Minio = Minio(
    endpoint=prd.S3_MINIO_ADDR,
    access_key=prd.S3_KEY,
    secret_key=prd.S3_SECRET,
    secure=True,
)

## Common Utils

In [ ]:
# =========================================
# Common Utils
# =========================================


class Actress(NamedTuple):
    # id is the unique id of the actress
    id: ObjectId
    # name is the name of the actress
    name: str
    # other_names is the other names of the actress, also contains the name
    other_names: List[str]
    # url is the url of the actress
    url: str


class Movie(NamedTuple):
    # actresses is a list of actresses' id
    actresses: List[ObjectId]
    # name is the code of the movie
    name: str
    # description is the title of the movie
    description: Optional[str]
    # img_urls is a list of image urls that describe the movie
    img_urls: List[str]
    # tags is a list of tags that describe the movie
    tags: List[str]
    # published_date is the date when the movie is published
    published_date: Optional[datetime]
    # url is the url of the movie
    urls: List[str]


_REGEX_SPLIT_ACTRESS_NAME = re.compile(r"([\w・]+)")


def split_actress_name(name: str) -> List[str]:
    """split actress name to list of names"""
    names = [
        v.strip().lower()
        for v in _REGEX_SPLIT_ACTRESS_NAME.findall(name)
        if v and v.strip()
    ]

    vals = []
    for v in names:
        vals.extend(v.split(","))

    names = [v.strip() for v in vals if v.strip()]
    names = list(set(names))
    return names


def save_actress(name: str, url: str, other_names: List[str]) -> Actress:
    """save actress to db, return actress_id"""
    logger.info(f"save actress {name}")

    url = url.strip()

    names = split_actress_name(name)
    other_names.extend(names)
    assert names, f"can not find name for {name=}, {url=}"

    name = choose_kanji_name(names)
    other_names.append(name)
    parsed_other_names: List[str] = []
    for v in other_names:
        parsed_other_names.extend(split_actress_name(v))
    parsed_other_names = list(set(parsed_other_names))

    col_actress.update_one(
        {"other_names": name},
        {
            "$set": {
                "name": name,
            },
            "$addToSet": {
                "other_names": {"$each": parsed_other_names},
                "urls": {"$each": [url]},
            },
        },
        upsert=True,
    )

    docu = col_actress.find_one({"other_names": name})
    assert docu, f"can not find actress {name}"
    return Actress(
        name=name,
        url=url,
        id=docu["_id"],
        other_names=docu["other_names"],
    )


def replace_image_url(movie_name: str, img_url: str) -> str:
    """download image and upload to s3, return new url"""
    docu = col_movies.find_one({"name": movie_name})
    if docu and docu["img_urls"]:
        return docu["img_urls"][0]

    if not img_url:
        return ""

    new_img_url = ""
    try:
        resp = requests.get(img_url)
        assert resp.status_code == 200, f"get image for {movie_name=}, {img_url=}"

        img_content = resp.content
        digest = sha256(img_content).hexdigest()
        objkey = f"jav/{digest[:2]}/{digest[2:4]}/{digest}.png"

        # check whether image exists
        new_img_url = f"{prd.S3_SERVER}/public/{objkey}"
        if requests.head(new_img_url).status_code == 200:
            return new_img_url

        logger.info(f"upload image to s3: {objkey}")
        s3cli.put_object(
            bucket_name="public",
            object_name=objkey,
            data=BytesIO(img_content),
            length=len(img_content),
            content_type="image/png",
        )
    except Exception:
        logger.exception(
            f"download and upload image got error for {movie_name=}, {img_url=}"
        )

    return new_img_url


REGEX_MOVIE_NAME = re.compile(r"^([A-Z]+)(\d+)$")


def normalize_movie_name(name: str) -> str:
    name = name.upper().strip()
    matched = REGEX_MOVIE_NAME.findall(name)
    if matched:
        prefix, num = matched[0]
        if prefix and num:
            name = f"{prefix}-{num}"

    return name


def save_movie(movie: Movie):
    """save movie to db by movie name"""
    movie_name = normalize_movie_name(movie.name)
    assert movie_name, f"movie_name is empty {movie.urls=}"

    img_urls = [v.strip() for v in movie.img_urls if v.strip()]
    tags = [v.strip() for v in movie.tags if v.strip()]
    actresses = [v for v in movie.actresses if v]
    movie_urls = [v.strip() for v in movie.urls if v.strip()]
    descriptions = [movie.description.strip()] if movie.description else []

    docu_to_set: Dict[str, Union[str, datetime]] = {"name": movie_name}
    if movie.published_date:
        docu_to_set["published_date"] = movie.published_date

    logger.info(f"save movie {movie_name=}")
    col_movies.update_one(
        {"name": movie_name},
        {
            "$set": docu_to_set,
            "$addToSet": {
                "actresses": {"$each": actresses},
                "tags": {"$each": tags},
                "img_urls": {"$each": img_urls},
                "urls": {"$each": movie_urls},
                "descriptions": {"$each": descriptions},
            },
        },
        upsert=True,
    )


# choose_kanji_name(["abc", "ブレイク・ブロッサム", "田中瞳", "ブレイク"])
def choose_kanji_name(names: List[str]) -> str:
    """Choose the kanji name from a list of names."""
    assert names, "names is empty"

    kanji_pattern = re.compile(r"[\u3040-\u30efa-zA-Z・]")
    hiragana_pattern = re.compile(r"[\u3040-\u309f]")
    english_pattern = re.compile(r"[a-zA-Z]")

    for name in names:
        if not kanji_pattern.search(name):
            # First, choose the name which contains Kanji.
            # not english nor hiragana
            return name

    for name in names:
        if hiragana_pattern.search(name):
            # Second, choose the name which contains Hiragana
            return name

    for name in names:
        if not english_pattern.search(name):
            # Last, choose the name without English alphabets
            return name

    # If none of the above conditions are met, return the first name
    return names[0]

## `https://onejav.com/`

In [ ]:
# =========================================
# https://onejav.com/
# =========================================

actress_index_url = "https://onejav.com/actress/"

REG_ONEJAV_ACTRESS_NAME = re.compile(r"([\w ]+)(?:<[^>]+>(\w+).*)?")


@timer
def _gen_onejav_actress(q: Queue[Optional[Actress]]):
    n_page = 1
    n_retry = 0
    while True:
        url = f"https://onejav.com/actress/?page={n_page}"
        logger.info(f"gen_onejav_actress get actress for {url=}")
        resp = requests.get(url)
        if resp.status_code == 500:
            if n_retry < 3:
                time.sleep(3)
                logger.warning(f"gen_onejav_actress retry {url=}, {n_retry=}")
                n_retry += 1
                continue
            else:
                n_retry = 0
                n_page += 1
                continue
        elif resp.status_code == 404:
            logger.info(f"gen_onejav_actress exit since for {url=}")
            return
        elif resp.status_code != 200:
            logger.info(
                f"gen_onejav_actress exit for {url=} [{resp.status_code}]{resp.text}"
            )
            return

        soup = BeautifulSoup(resp.text, "html.parser")
        for card in soup.select(".container .card"):
            name, url = "", ""

            ele = card.select_one(".card-header .card-header-title")
            assert ele, f"cannot find actress name for {url=}"
            name = ele.decode_contents().strip()

            ele = card.select_one("a")
            assert ele, f"cannot find actress url for {url=}"
            url = f"https://onejav.com{ele.attrs['href']}"

            # parse actress name
            # name = 'Jun Suehiro <small class="text-muted ml-1">末広純</small>'
            [(alt_name, name)] = REG_ONEJAV_ACTRESS_NAME.findall(name)
            name = name.strip()
            alt_name = alt_name.strip()
            name = name or alt_name

            logger.debug(f"yield actress {name}")
            actress = save_actress(name, url, list(set([name, alt_name])))
            q.put(actress)

        n_page += 1



@timer
def _gen_onejav_movies_by_actress(upstream_q: Queue[Optional[Actress]]):
    while True:
        actress = upstream_q.get()
        if actress is None:
            upstream_q.put(None)
            logger.info(f"gen_onejav_movies_by_actress exit")
            return

        n_page = 1
        n_retry = 0
        while True:
            url = f"{actress.url}?page={n_page}"
            resp = requests.get(url)
            if resp.status_code == 404:
                logger.info(f"gen_movies got 404 for {url=}")
                break
            elif resp.status_code != 200:
                if n_retry < 3:
                    time.sleep(3)
                    logger.warning(f"gen_movies retry {url=}, {n_retry=}")
                    n_retry += 1
                    continue
                else:
                    n_retry = 0
                    n_page += 1
                    logger.warning(f"gen_movies skip for {url=}, {n_retry=}")
                    continue

            soup = BeautifulSoup(resp.text, "html.parser")
            for card in soup.select(".container .card"):
                try:
                    ele = card.select_one(".image")
                    assert ele, f"can not find image {url=}"
                    img_url = ele.attrs["src"]

                    ele = card.select_one(".title a")
                    assert ele, f"can not find title {url=}"
                    name = normalize_movie_name(ele.decode_contents())

                    description = None
                    if card.select_one(".has-text-grey-dark"):
                        ele = card.select_one(".has-text-grey-dark")
                        assert ele, f"can not find description {url=}"
                        description = ele.decode_contents().strip()
                    tags = [
                        ele.decode_contents().strip()
                        for ele in card.select(".tags .tag")
                    ]

                    date: Optional[datetime] = None
                    ele = card.select_one(".card-content .title .subtitle a")
                    if ele and ele.attrs.get("href"):
                        # /2024/07/14
                        date = datetime.strptime(ele.attrs["href"], "/%Y/%m/%d")

                    img_url = replace_image_url(name, img_url)
                    movie = Movie(
                        actresses=[actress.id],
                        name=name,
                        description=description,
                        img_urls=[img_url] if img_url else [],
                        tags=tags,
                        published_date=date,
                        urls=[],
                    )
                    save_movie(movie)
                except Exception:
                    logger.exception(f"parse movie error {url=}")
                    continue

            n_page += 1


@timer
def fetch_onejav_by_actress_list():
    """fetch actresses and movies by actress list and save to db"""
    actress_queue = Queue(maxsize=10)

    fs_load_actress = []
    fs_load_movies = []

    fs_load_actress.append(executor.submit(_gen_onejav_actress, actress_queue))
    for _ in range(20):
        fs_load_movies.append(
            executor.submit(_gen_onejav_movies_by_actress, actress_queue)
        )

    wait(fs_load_actress, return_when=ALL_COMPLETED)
    actress_queue.put(None)
    wait(fs_load_movies, return_when=ALL_COMPLETED)

## `https://jav.land/`

In [ ]:
# =========================================
# https://jav.land/
# =========================================

actress_index_url = "https://jav.land/ja/star_list.php?prefix={alphabet}&page={page}"

REGEX_JAVLAND_ACTRESS_NAME = re.compile(r"(\w+)(?:（(\w+)(?:、(\w+){0,}）))?")


class JavlandMovieInfo(NamedTuple):
    actress: Actress
    url: str
    description: Optional[str]


@timer
def gen_javland_actress(q: Queue[Optional[Actress]]):
    n_page = 1
    alphabet = "A"
    n_retry = 0
    while alphabet <= "Z":
        logger.info(f"gen_javland_actress for {alphabet=}")
        while True:
            url = actress_index_url.format(alphabet=alphabet, page=n_page)
            try:
                logger.info(f"request actress {url=}")
                resp = requests.get(url, timeout=10)
                if resp.status_code % 100 == 5:
                    if n_retry < 3:
                        time.sleep(3)
                        logger.warning(f"gen_javland_actress retry {url=}, {n_retry=}")
                        n_retry += 1
                        continue
                    else:
                        n_retry = 0
                        n_page += 1
                        continue
                elif resp.status_code == 404:
                    logger.info(f"gen_javland_actress exit since for {url=}")
                    break
                elif resp.status_code != 200:
                    logger.info(
                        f"gen_javland_actress exit for {url=} [{resp.status_code}]{resp.text}"
                    )
                    break

                soup = BeautifulSoup(resp.text, "html.parser")
                cards = soup.select(".container-fluid .genre_list a.text-center")
                if not cards:
                    break

                for card in cards:
                    raw_name = card.decode_contents().strip()

                    # parse actress name
                    # name = "馬場ふみか" / ayami（赤西涼、まひる）
                    matched = REGEX_JAVLAND_ACTRESS_NAME.findall(raw_name)
                    assert matched, f"can not parse actress name {raw_name=}"
                    assert matched[0], f"can not parse actress name {raw_name=}"
                    other_names = list(
                        set([v.strip() for v in matched[0] if v.strip()])
                    )
                    name = choose_kanji_name(other_names)
                    assert name, f"can not parse actress name {raw_name=}"

                    actress_url = f"https://jav.land{card.attrs['href']}"

                    logger.debug(f"yield actress {name}")
                    actress = save_actress(name, actress_url, other_names)
                    q.put(actress)
            except Exception:
                time.sleep(1)
                logger.exception(f"gen_javland_actress error {url=}")
                continue

            n_page += 1

        n_page = 1
        alphabet = chr(ord(alphabet) + 1)


@timer
def gen_javland_movies_by_actress(
    upstream_q: Queue[Optional[Actress]],
    downstream_q: Queue[Optional[JavlandMovieInfo]],
):
    while True:
        actress = upstream_q.get()
        if actress is None:
            upstream_q.put(None)
            logger.info(f"gen_javland_movies_by_actress exit")
            return

        n_page = 1
        n_retry = 0
        while True:
            url = f"{actress.url}?page={n_page}"
            try:
                logger.info(f"request movies for {url=}")
                resp = requests.get(url, timeout=10)
                if resp.status_code % 100 == 5:
                    if n_retry < 3:
                        time.sleep(3)
                        logger.warning(f"gen_movies retry {url=}, {n_retry=}")
                        n_retry += 1
                        continue
                    else:
                        n_retry = 0
                        n_page += 1
                        continue
                elif resp.status_code == 404:
                    logger.info(f"gen_movies exit since for {url=}")
                    break
                elif resp.status_code != 200:
                    logger.warning(
                        f"gen_movies skip since for {url=} [{resp.status_code}]{resp.text}"
                    )
                    n_page += 1
                    continue

                soup = BeautifulSoup(resp.text, "html.parser")
                cards = soup.select(".container-fluid .videothumblist .video.panel")
                if not cards:
                    logger.info(f"can not find cards for {url=}")
                    break

                for card in cards:
                    ele = card.select_one(".panel-footer a")
                    assert ele, f"can not find url {url=}"
                    url = f"https://jav.land{ele.attrs['href']}"
                    description = ele.decode_contents().strip()

                    downstream_q.put(
                        JavlandMovieInfo(
                            actress=actress, url=url, description=description
                        )
                    )

                n_page += 1
            except:
                time.sleep(1)
                logger.exception(f"gen_javland_movies_by_actress got error for {url=}")


@timer
def fetch_javland_movie(q: Queue[Optional[JavlandMovieInfo]]):
    while True:
        movie_info = q.get()
        if movie_info is None:
            q.put(None)
            logger.info(f"fetch_javland_movie exit")
            return

        nretry = 0
        while nretry < 3:
            try:
                logger.debug(f"fetch movie for {movie_info.url=}")
                resp = requests.get(
                    movie_info.url,
                    timeout=10,
                    headers={
                        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
                        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
                    },
                )
                if resp.status_code != 200:
                    logger.warning(
                        f"fetch_javland_movie error, {movie_info.url=} [{resp.status_code}]{resp.text}"
                    )
                    time.sleep(3)
                    nretry += 1
                    continue

                soup = BeautifulSoup(resp.text, "html.parser")

                name: str = ""
                tags: List[str] = []
                date: Optional[datetime] = None
                rows = soup.select(".k-right .videotextlist tr")
                for row in rows:
                    ele = row.select_one("td:nth-child(1)")
                    key: str = ""
                    value: str = ""

                    if ele:
                        key = ele.decode_contents().strip()
                    else:
                        continue

                    ele = row.select_one("td:nth-child(2)")
                    if ele:
                        value = ele.decode_contents().strip()
                    else:
                        continue

                    if "DVD ID" in key:
                        name = normalize_movie_name(value)
                    elif "ジャンル" in key:
                        vals = row.select("td:nth-child(2) .genre a")
                        tags = list(
                            set([val.decode_contents().strip() for val in vals])
                        )
                    elif "発売日" in key:
                        date = datetime.strptime(value, "%Y-%m-%d")

                ele = soup.select_one(".k-right img.img-responsive")
                assert ele, f"can not find image {movie_info.url=}"
                img_url = ele.attrs["src"]
                img_url = replace_image_url(name, img_url)

                movie = Movie(
                    actresses=[movie_info.actress.id],
                    name=name,
                    description=movie_info.description,
                    img_urls=[img_url] if img_url else [],
                    tags=tags,
                    published_date=date,
                    urls=[movie_info.url],
                )
                save_movie(movie)
                break
            except Exception:
                nretry += 1
                time.sleep(3)
                logger.exception(f"fetch_javland_movie error {movie_info.url=}")
                continue


@timer
def fetch_javland_by_actress_list():
    """fetch actresses and movies by actress list and save to db"""
    actress_queue = Queue(maxsize=10)
    movie_queue = Queue(maxsize=10)

    fs_load_actress = []
    fs_load_movies = []
    fs_save_movies = []

    fs_load_actress.append(executor.submit(gen_javland_actress, actress_queue))
    for _ in range(10):
        fs_load_movies.append(
            executor.submit(gen_javland_movies_by_actress, actress_queue, movie_queue)
        )
        fs_save_movies.append(executor.submit(fetch_javland_movie, movie_queue))

    wait(fs_load_actress, return_when=ALL_COMPLETED)
    actress_queue.put(None)
    wait(fs_load_movies, return_when=ALL_COMPLETED)
    movie_queue.put(None)
    wait(fs_save_movies, return_when=ALL_COMPLETED)

In [ ]:
# system_prompt = dedent(
#     """
#     You are a tokenization engine. Follow these instructions sequentially, providing the output of each step directly as requested:

#     1. **Keyword Extraction:** Identify and extract the most important keywords from the user's input, based on your understanding. Output these keywords separated by commas.

#     2. **Expand JSON Array with Translations:** Take the provided JSON array and create an expanded version. The original array should remain intact. For each non-English word in the array, translate it into English and append the translation to the end of the array.

#     3. **Output Format:** Present the final expanded JSON array strictly in the format below, without including any additional text, explanations, or characters:

#         `result: ["word-1", "word-2", ...]`
#     """
# )
# user_prompt = "焦らし寸止め絶頂セックス あやみ史上1番エロいです！あやみはまだまだ進化しています！ ACT.03 あやみ旬果"

# segment_word(user_prompt)

## `https://javtrailers.com/`

In [ ]:
# =========================================
# https://javtrailers.com/search/ipzz-056
# =========================================


@timer
def _list_javtrailers_movies(q: Queue[Optional[str]]):
    """scan movies by page and put url to queue"""
    n_page = JAVTRAILERS_START_PAGE
    n_retry = 0
    while True:
        url = f"https://javtrailers.com/ja/videos?page={n_page}"
        try:
            logger.info(f"list_movies get movies for {url=}")
            resp = requests.get(
                url,
                timeout=10,
                headers={
                    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
                    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
                },
            )
            assert (
                resp.status_code == 200
            ), f"get movies got [{resp.status_code}]{resp.text}"
            if "No videos available, check out these popular videos" in resp.text:
                logger.info(f"list_movies exit since for find no videos for {url=}")
                return

            soup = BeautifulSoup(resp.text, "html.parser")
            cards = soup.select("#home .videos-list .card-container a")
            for card in cards:
                url = f"https://javtrailers.com{card.attrs['href']}"
                logger.debug(f"yield movie {url=}")
                q.put(url)

            n_page += 1
        except Exception:
            time.sleep(3)
            logger.warning(f"list_movies retry {url=}, {n_retry=}")
            n_retry += 1
            if n_retry > 3:
                n_retry = 0
                n_page += 1
            continue


REGEX_JAVTRAILER_ROW_VAL = re.compile(r"</span>([^<]+)</p>")


@timer
def _fetch_javtrailers_movie_detail(
    ups_q: Queue[Optional[str]], downs_q: Queue[Optional[Movie]]
):
    while True:
        url = ups_q.get()
        if url is None:
            ups_q.put(None)
            logger.info(f"fetch_movie_detail exit")
            return

        n_retry = 0
        while n_retry < 3:
            try:
                logger.info(f"fetch movie detail for {url=}")
                resp = requests.get(
                    url,
                    timeout=10,
                    headers={
                        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
                        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
                    },
                )
                assert (
                    resp.status_code == 200
                ), f"fetch movie detail got [{resp.status_code}]{resp.text}"

                soup = BeautifulSoup(resp.text, "html.parser")

                ele = soup.select_one("#description .lead")
                description = ele.decode_contents().strip() if ele else ""

                name: str = ""
                published_date: Optional[datetime] = None
                tags: List[str] = []
                actresses: List[ObjectId] = []

                rows = soup.select("#description #info-row p")
                # [<p class="mb-1"><span class="font-weight-bold mr-3">DVD ID:</span> EBWH-090</p>,
                # <p class="mb-1"><span class="font-weight-bold mr-3">Content ID:</span> ebwh00090</p>,
                # <p class="mb-1"><span class="font-weight-bold mr-3">Release Date:</span> 12 Apr 2024</p>,
                # <p class="mb-1"><span class="font-weight-bold mr-3">Duration:</span> 120 mins</p>,
                # <p class="mb-1"><span class="font-weight-bold mr-3">Director:</span> Inavar イナバール</p>,
                for row in rows:
                    ele = row.select_one("span")
                    key = ele.decode_contents().strip() if ele else None
                    if not key:
                        continue

                    val: str = ""
                    matched = REGEX_JAVTRAILER_ROW_VAL.findall(row.decode().strip())
                    if matched:
                        val = matched[0].strip()
                    else:
                        logger.debug(f"can not find value for {key=}, {row.decode()=}")

                    if val and "DVD ID" in key:
                        name = normalize_movie_name(val)
                    elif val and "Release Date" in key or "商品発売日" in key:
                        # 12 Apr 2024
                        published_date = datetime.strptime(val, "%d %b %Y")
                    elif "Categories" in key or "ジャンル" in key:
                        tags = [
                            ele.decode_contents().strip().lower()
                            for ele in row.select("a")
                        ]
                    elif "Cast(s)" in key or "出演者" in key:
                        fs = []
                        for actress_ele in row.select("a"):
                            actress_name = actress_ele.decode_contents().strip()
                            # actress = save_actress(
                            #     name=actress_name,
                            #     url=f"https://javtrailers.com{actress_ele.attrs['href']}",
                            #     other_names=[actress_name],
                            # )
                            # actresses.append(actress.id)
                            fs.append(
                                executor.submit(
                                    save_actress,
                                    actress_name,
                                    f"https://javtrailers.com{actress_ele.attrs['href']}",
                                    [actress_name],
                                )
                            )

                        wait(fs, return_when=ALL_COMPLETED)
                        for f in fs:
                            actress = f.result()
                            actresses.append(actress.id)

                assert name, f"can not find movie name for {url=}"
                if not actresses:
                    logger.warning(f"can not find actress for {name=}, {url=}")

                ele = soup.select("#description img")
                img_url = ele[0].attrs["src"].replace("ps.", "pl.") if ele else ""
                img_url = replace_image_url(name, img_url)

                movie = Movie(
                    actresses=actresses,
                    name=name,
                    description=description,
                    img_urls=[img_url] if img_url else [],
                    tags=tags,
                    published_date=published_date,
                    urls=[url],
                )
                downs_q.put(movie)
                break
            except Exception:
                time.sleep(3)
                logger.exception(f"fetch movie detail got error for {url=}")
                n_retry += 1
                continue


@timer
def _save_javtrailers_movie(q: Queue[Optional[Movie]]):
    while True:
        movie = q.get()
        if movie is None:
            q.put(None)
            logger.info(f"save_movie exit")
            return

        save_movie(movie)


@timer
def fetch_javtrailers():
    """fetch movies from javtrailers and save to db"""
    movie_list_queue: Queue[Optional[str]] = Queue(maxsize=50)
    movie_details_queue: Queue[Optional[Movie]] = Queue(maxsize=50)

    fs_list_movies = []
    fs_fetch_movies = []
    fs_save_movies = []

    fs_list_movies.append(executor.submit(_list_javtrailers_movies, movie_list_queue))
    for _ in range(10):
        fs_fetch_movies.append(
            executor.submit(
                _fetch_javtrailers_movie_detail, movie_list_queue, movie_details_queue
            )
        )
        fs_save_movies.append(
            executor.submit(_save_javtrailers_movie, movie_details_queue)
        )

    wait(fs_list_movies, return_when=ALL_COMPLETED)
    movie_list_queue.put(None)
    wait(fs_fetch_movies, return_when=ALL_COMPLETED)
    movie_details_queue.put(None)
    wait(fs_save_movies, return_when=ALL_COMPLETED)


# fetch_javtrailers()

## Fulltext Search

In [ ]:
# =========================================
# generate full text search metadata for each movie
# =========================================

def llm_oneshot(model: str, system_prompt: str, user_prompt: str) -> str:
    """get response from LLM"""
    url = f"{prd.OPENAI_API}/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {prd.OPENAI_TOKEN}",
    }
    body = {
        "model": model,
        "max_tokens": 3000,
        "stream": False,
        "messages": [
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": user_prompt,
            },
        ],
    }

    resp = requests.post(url, headers=headers, json=body)
    assert resp.status_code == 200, f"[{resp.status_code}]{resp.text}"

    return resp.json()["choices"][0]["message"]["content"]


GEMMA_API = "http://100.92.237.35:7861/chat/completions"


def llm_oneshot_gemma(system_prompt: str, user_prompt: str) -> str:
    body = {
        "model": "gemma-2b-it",
        "max_tokens": 500,
        "stream": False,
        "messages": [
            # gemma does not support `system_prompt`
            {"role": "user", "content": f"{system_prompt}\n>>\n{user_prompt}"},
        ],
    }
    resp = requests.post(GEMMA_API, json=body)
    assert resp.status_code == 200, f"[{resp.status_code}]{resp.text}"
    return resp.json()["completion"]


REGEX_SEGMENT_RESULT = re.compile(
    r"(?:result|keywords|keyword|words|translated|translations|expanded)\"?: (\[[^\]\[]*\])",
    flags=re.IGNORECASE|re.MULTILINE
)


def segment_word(sentence: str) -> List[str]:
    """segment sentence to words"""
    if not ENABLE_FULLTEXT_LLM:
        return []

    system_prompt = dedent(
        """
        You act as a sophisticated tokenization engine. Follow these instructions step by step, executing each step in order and return every step's output:

        1. Analyze the user's input text and extract the most significant keywords based on semantic importance. Output these keywords as a comma-separated list.

        2. Take the keyword list from step 1 and create a JSON array. And append every keyword's English translations to the end of the array. Please ensure the original words are retained.

        3. Return the final result as a JSON array using this exact format, without any additional text or characters. Here is an example of last step output:

            ```json
            result: ["word-1", "word-2", "english-1", "english-2"]
            ```
        """
    )
    user_prompt = sentence

    # resp = llm_oneshot("deepseek-coder", system_prompt, user_prompt)
    resp = llm_oneshot_gemma(system_prompt, user_prompt)
    logger.debug(f"segment_word got llm resp {resp=}")

    matched = REGEX_SEGMENT_RESULT.findall(resp)
    matched = [val for val in matched if 'word-1' not in val]
    assert matched, f"invalid format of response {resp=}"

    words: List[str] = []
    for match in matched:
        if 'word-1' in match:
            continue

        try:
            vs = json.loads(match)
            assert isinstance(vs, list)
        except Exception:
            logger.warning(f"cannot parse LLM's response {match=}")
            continue

        words.extend(vs)

    # additions text split
    new_words = []
    for word in words:
        new_words.extend(word.split(","))
    words = new_words

    words = [word.strip() for word in words if isinstance(word, str) and word.strip()]
    words = [word for word in words if word not in ["word-1", "word-2", "english-1", "english-2"]]
    words = [word for word in words if len(word) <= 20]
    words = list(set(words))

    logger.debug(f"segment_word result {words=}")
    return words


@timer
def _list_movies_in_db(q: Queue[Movie]):
    for docu in col_movies.find().sort([("_id", pymongo.DESCENDING)]):
        movie = Movie(
            actresses=docu["actresses"],
            name=docu["name"],
            description=','.join(docu.get("descriptions", [])),
            img_urls=docu["img_urls"],
            tags=docu["tags"],
            published_date=docu.get("published_date"),
            urls=docu.get("urls"),
        )
        q.put(movie)


class FulltextItem(NamedTuple):
    movie_id: ObjectId
    word: str


@timer
def _generate_fulltext_for_movie(
    upstream_q: Queue[Optional[Movie]], downstream_q: Queue[FulltextItem]
):
    """generate fulltext for each movie, each word points to a movie"""
    while True:
        try:
            movie = upstream_q.get()
            if movie is None:
                upstream_q.put(None)
                logger.info(f"_generate_fulltext_for_movie exit")
                return

            keywords: List[str] = [movie.name]
            keywords.extend(movie.tags)

            # load movie
            logger.info(f"_generate_fulltext_for_movie for {movie.name=}")
            movie_docu = col_movies.find_one({"name": movie.name})
            assert movie_docu, f"can not find movie {movie.name}"

            # load actress
            for docu in col_actress.find({"_id": {"$in": movie_docu["actresses"]}}):
                keywords.extend(docu.get("other_names", []))

            keywords = list(set(keywords))

            tobe_segment = f"{','.join(keywords)}, {','.join(movie_docu.get('descriptions', []))}"
            movie_words = segment_word(tobe_segment)
            keywords.extend(movie_words)

            keywords = list(set(keywords))
            for word in keywords:
                downstream_q.put(FulltextItem(movie_id=movie_docu["_id"], word=word))
        except Exception:
            logger.exception(f"error in _generate_fulltext_for_movie")


@timer
def _save_fulltext_item(q: Queue[Optional[FulltextItem]]):
    """save fulltext item to db, each word may points to multiple movies"""
    last_movie_id: Optional[ObjectId] = None
    while True:
        try:
            item = q.get()
            if item is None:
                q.put(None)
                logger.info(f"_save_fulltext_item exit")
                return

            if item.movie_id != last_movie_id:
                col_movies.update_one(
                    {"_id": last_movie_id},
                    {"$set": {"fulltext_updated_at": datetime.now()}},
                )
                last_movie_id = item.movie_id

            # each word may points to multiple movies,
            # these movies are stored in a set
            col_fulltext.update_one(
                {"word": item.word},
                {"$addToSet": {"movies": item.movie_id}},
                upsert=True,
            )
        except Exception:
            logger.exception(f"error in _save_fulltext_item")


@timer
def generate_fulltext():
    movie_queue = Queue(maxsize=100)
    fulltext_queue = Queue(maxsize=100)

    fs_list_movies = []
    fs_generate_fulltext = []
    fs_save_fulltext = []

    fs_list_movies.append(executor.submit(_list_movies_in_db, movie_queue))
    for i in range(10):
        fs_generate_fulltext.append(
            executor.submit(_generate_fulltext_for_movie, movie_queue, fulltext_queue)
        )

    for i in range(20):
        fs_save_fulltext.append(executor.submit(_save_fulltext_item, fulltext_queue))

    wait(fs_list_movies, return_when=ALL_COMPLETED)
    movie_queue.put(None)
    wait(fs_generate_fulltext, return_when=ALL_COMPLETED)
    fulltext_queue.put(None)
    wait(fs_save_fulltext, return_when=ALL_COMPLETED)

In [ ]:
REGEX_MOVIE_NAME = re.compile(r"^([A-Z]+)(\d+)$")


@timer
def convert_movies():
    # remove empty images
    for docu in col_movies.find({"img_urls": ""}):
        imgs = [v for v in docu["img_urls"] if v.strip()]
        col_movies.update_one(
            {"_id": docu["_id"]},
            {"$set": {"img_urls": imgs}},
        )


@timer
def remove_long_words():
    result = col_fulltext.delete_many({"$expr": {"$gt": [{"$strLenCP": "$word"}, 20]}})
    logger.info(f"remove_long_words {result.deleted_count=}")


@timer
def _scan_movies(q: Queue[Optional[Dict[str, Any]]]):
    for docu in col_movies.find({"description": {"$exists": True}}):
        q.put(docu)

    q.put(None)


@timer
def _scan_actress(q: Queue[Optional[Dict[str, Any]]]):
    # for docu in col_actress.find({"name": {"$regex": r"[（\(]"}}):
    for docu in col_actress.find({"name": ""}):
        # for docu in col_actress.find():
        q.put(docu)

    q.put(None)


@timer
def _do_normalize_actress_name(q: Queue[Optional[Dict[str, Any]]]):
    while True:
        actress_docu = q.get()
        if actress_docu is None:
            q.put(None)
            logger.info(f"_do_normalize_actress_name exit")
            return

        name = actress_docu["name"]
        other_names = actress_docu.get("other_names", [])

        names = split_actress_name(name)
        other_names.extend(names)
        assert names, f"can not find name for {name=}"

        name = choose_kanji_name(names)
        other_names.append(name)
        parsed_other_names: List[str] = []
        for v in other_names:
            parsed_other_names.extend(split_actress_name(v))
        parsed_other_names = list(set(parsed_other_names))

        col_actress.update_one(
            {"_id": actress_docu["_id"]},
            {
                "$set": {"name": name, "other_names": parsed_other_names},
            },
        )


@timer
def normalize_actress_name():
    actress_queue = Queue(maxsize=10)
    fs_scan_actress = []
    fs_convert_actress = []

    fs_scan_actress.append(executor.submit(_scan_actress, actress_queue))
    for i in range(40):
        fs_convert_actress.append(
            executor.submit(_do_normalize_actress_name, actress_queue)
        )

    wait(fs_scan_actress, return_when=ALL_COMPLETED)
    actress_queue.put(None)
    wait(fs_convert_actress, return_when=ALL_COMPLETED)


@timer
def _convert_movie(q: Queue[Optional[Dict[str, Any]]]):
    while True:
        movie_docu = q.get()
        if movie_docu is None:
            q.put(None)
            logger.info(f"_convert_movie exit")
            return

        col_movies.update_one(
            {"_id": movie_docu["_id"]},
            {
                "$set": {"descriptions": [movie_docu["description"]]},
                "$unset": {"description": ""},
            },
        )


_duplicate_actress_lock = RLock()
_duplicate_actress_docu_lock: Dict[str, RLock] = {}


@timer
def _do_reset_actress_name(q: Queue[Optional[Dict[str, Any]]]):
    while True:
        actress_docu = q.get()
        if actress_docu is None:
            q.put(None)
            logger.info(f"_do_reset_actress_name exit")
            return

        other_names = [v for v in actress_docu["other_names"] if v.strip()]
        name = choose_kanji_name(other_names)
        col_actress.update_one(
            {"_id": actress_docu["_id"]},
            {
                "$set": {
                    "name": name,
                    "other_names": other_names,
                },
            },
        )


@timer
def reset_actress_name():
    actress_queue = Queue(maxsize=40)
    fs_scan_actress = []
    fs_convert_actress = []

    fs_scan_actress.append(executor.submit(_scan_actress, actress_queue))
    for i in range(40):
        fs_convert_actress.append(
            executor.submit(_do_reset_actress_name, actress_queue)
        )

    wait(fs_scan_actress, return_when=ALL_COMPLETED)
    actress_queue.put(None)
    wait(fs_convert_actress, return_when=ALL_COMPLETED)


@timer
def _deal_duplicate_actress(q: Queue[Optional[Dict[str, Any]]]):
    while True:
        actress_docu = q.get()
        if actress_docu is None:
            q.put(None)
            logger.info(f"_deal_duplicate_actress exit")
            return

        all_docus = [actress_docu]
        for name in actress_docu["other_names"]:
            all_docus.extend(col_actress.find({"other_names": name}))

        earliest_docu = sorted(all_docus, key=lambda x: x["_id"])[0]
        tobe_delete_docus = [
            docu for docu in all_docus if docu["_id"] != earliest_docu["_id"]
        ]

        if not tobe_delete_docus:
            continue

        with _duplicate_actress_lock:
            lock = _duplicate_actress_docu_lock.get(earliest_docu["_id"], RLock())

        with lock:
            for docu in all_docus:
                earliest_docu.update(
                    {
                        "urls": earliest_docu.get("urls", []) + docu.get("urls", []),
                        "other_names": earliest_docu["other_names"]
                        + docu["other_names"],
                    }
                )

                # replace actress id in movies by earliest actress id
                if docu["_id"] != earliest_docu["_id"]:
                    col_movies.update_many(
                        {"actresses": docu["_id"]},
                        {
                            "$addToSet": {"actresses": earliest_docu["_id"]},
                        },
                    )

            earliest_docu["urls"] = list(set(earliest_docu["urls"]))
            earliest_docu["other_names"] = [
                v for v in earliest_docu["other_names"] if v.strip()
            ]
            earliest_docu["other_names"] = list(set(earliest_docu["other_names"]))
            earliest_docu["name"] = choose_kanji_name(earliest_docu["other_names"])

            # print(f"{earliest_docu=}")
            # return

            # print(f'{earliest_docu["_id"]}')
            # print(f"{tobe_delete_docus=}")
            # return

            # update actress
            col_actress.update_one(
                {"_id": earliest_docu["_id"]},
                {
                    "$set": {
                        "name": earliest_docu["name"],
                    },
                    "$addToSet": {
                        "urls": {"$each": earliest_docu["urls"]},
                        "other_names": {"$each": earliest_docu["other_names"]},
                    },
                },
            )

            # delete duplicate actress in movies
            col_movies.update_many(
                {"actresses": {"$in": [docu["_id"] for docu in tobe_delete_docus]}},
                {
                    "$pull": {
                        "actresses": {
                            "$in": [docu["_id"] for docu in tobe_delete_docus]
                        }
                    }
                },
            )

            # db.movies.update_many(
            #     {"actresses": {"$in": [ObjectId("66becdb7c59907294115f007")]}},
            #     {"$pull": {"actresses": {"$in": [ObjectId("66becdb7c59907294115f007")]}}},
            # )

            # delete duplicate actress
            col_actress.delete_many(
                {"_id": {"$in": [docu["_id"] for docu in tobe_delete_docus]}}
            )

            logger.info(
                f"gather {len(tobe_delete_docus)} duplicate actresses into {earliest_docu['name']=}, {earliest_docu['_id']=}"
            )

            with _duplicate_actress_lock:
                try:
                    del _duplicate_actress_docu_lock[earliest_docu["_id"]]
                except KeyError:
                    pass


@timer
def remove_duplicate_actress():
    actress_queue = Queue(maxsize=10)
    fs_scan_actress = []
    fs_convert_actress = []

    fs_scan_actress.append(executor.submit(_scan_actress, actress_queue))
    for i in range(50):
        fs_convert_actress.append(
            executor.submit(_deal_duplicate_actress, actress_queue)
        )

    wait(fs_scan_actress, return_when=ALL_COMPLETED)
    actress_queue.put(None)
    wait(fs_convert_actress, return_when=ALL_COMPLETED)


@timer
def convert_movie_description_to_array():
    movie_queue = Queue(maxsize=10)
    fs_scan_movies = []
    fs_convert_movie = []

    fs_scan_movies.append(executor.submit(_scan_movies, movie_queue))
    for i in range(20):
        fs_convert_movie.append(executor.submit(_convert_movie, movie_queue))

    wait(fs_scan_movies, return_when=ALL_COMPLETED)
    wait(fs_convert_movie, return_when=ALL_COMPLETED)


# convert_movie_description_to_array()
# reset_actress_name()
remove_duplicate_actress()
# normalize_actress_name()